In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import cv2
from PIL import Image
from numpy import *
import imutils
import freenect
import tkinter as tk
import keras
from keras.models import load_model

Using TensorFlow backend.


In [2]:
labels_name = { 0:'0', 1:'1',  2:'2',  3:'3',  4:'4', 5:'5',  6:'6',  7:'7',  8:'8', 9:'9' , 10:'A', 11:'B',
              12:'C', 13:'D', 14:'E', 15:'F', 16:'G', 17:'H', 18:'I', 19:'J', 20:'K', 21:'L', 22:'M', 23:'N',
              24:'O', 25:'P',26: 'Q', 27:'R',28: 'S', 29:'T', 30:'U', 31:'V', 32:'W', 33:'X', 34:'Y', 35:'Z'}

In [3]:
# load model
model = load_model('final_RGB_MODEL_&_ARCHITECTURE.h5')
# summarize model.
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 61, 91, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 61, 91, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 45, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 43, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 28, 43, 32)        0         
_______________________________________

In [4]:
def get_video():
    array,_ = freenect.sync_get_video()
    arr = array.astype(np.uint8)
    array = cv2.cvtColor(arr,cv2.COLOR_RGB2BGR)
    return array

def get_depth_segmented():
    array,_ = freenect.sync_get_depth_reg()
    arr = array.astype(np.uint8)
    arr = np.where( array > 750, arr,0).astype(np.uint8)
    array = np.where( array < 1000, arr,0).astype(np.uint8)
    return array

def get_depth():
    array,_ = freenect.sync_get_depth_reg()
    arr = array.astype(np.uint8)
    return arr


def predict(img):
    input_image = img
    img_cols = 93
    img_rows = 63
    input_image = cv2.resize(input_image,(img_cols,img_rows),interpolation = cv2.INTER_AREA)
    img_list = []
    img_list.append(input_image)
    img_list = np.array( img_list, dtype=np.float32)
    img_list = np.multiply(img_list, 1.0 / 255.0)
    m = model.predict_classes(img_list)
    return m

In [6]:
if __name__ == "__main__":  
    
    while 1:
        frame = get_video()
        frame = frame[135:345,165:475]
        depth = get_depth()
        depth = depth[135:345,165:475]
        depth_seg = get_depth_segmented()
        depth_seg = depth_seg[135:345,165:475]
        ret , mask = cv2.threshold(depth_seg, 0, 255, cv2.THRESH_BINARY)
        #mask_inv = cv2.bitwise_not(mask)
        img1_bg = cv2.bitwise_and(frame,frame,mask = mask)
        
        #depth_seg = cv2.cvtColor(depth_seg,cv2.COLOR_BGR2RGB)
        m = predict(img1_bg)
        
        text = (labels_name[m[0]])
        txt = str(text)
        img = np.zeros((512,512,3), np.uint8)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img,txt,(195,300), font, 8,(255,255,255),10,cv2.LINE_AA)
        
        img1_bg = cv2.resize(img1_bg,(465,315),interpolation = cv2.INTER_AREA)
        
        cv2.imshow('RGB',frame)
        cv2.imshow('Depth Segmented',depth_seg)
        cv2.imshow("rgb segmented", img1_bg)
        cv2.imshow("depth", depth)
        cv2.imshow('result', img)
        
        # quit program when 'esc' key is pressed
        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break
        #if k ==  ord('s'):
            #f = 2
            #cv2.imwrite("/home/paranjoy/Documents/t6.jpg" , img1_bg)
            #break

    cv2.destroyAllWindows()

In [16]:
img1_bg.shape

(210, 310, 3)

In [17]:
depth_seg.shape

(210, 310)